In [1]:
from utils import *

In [2]:
generate_inventory(verbose=True)

Total Monies: 817.65
Saving inventory data with (121, 6) shape


In [3]:
def analyse_stocktake_value():
    df = pd.read_parquet('intermediate/inventory.parquet')

    item_count = df.groupby('item')['count'].sum()
    items = load_items()
    # Adjust for self demand
    for item, detail in items.items():
        if 'self_demand' in detail:
            for character, count in detail['self_demand'].items():
                if item in item_count.index:
                    item_count.loc[item] -= count

    ####

    # Get prices from below
    prices.name = 'price'
    stocktake = pd.DataFrame(item_count).join(prices)
    stocktake['est'] = stocktake['count'] * stocktake['price']
    print(int(stocktake['est'].sum() / 10000))                

In [4]:
get_character_needs(['Melue', 'Zegna'])

{'Melue': {'Elixir of Giants': -1}, 'Zegna': {}}

In [5]:
generate_auction_history(verbose=True)

12767 auction events
9900 auction events of interest


In [ ]:
def get_profits(date=None, item=None):
    df_raw = pd.read_parquet('intermediate/auctions.parquet')
    df_raw = df_raw.drop_duplicates(subset=['auction_type','item', 'timestamp'])

    df_failed = df_raw[df_raw['auction_type'].isin(['failed'])]
    df_success = df_raw[df_raw['auction_type'].isin(['buy_price', 'sell_price'])]
    df_auction_type = df_success.set_index(['auction_type','item', 'timestamp'])['price_per'].unstack().T

    item_history = df_success.set_index(['item','auction_type', 'timestamp'])['price_per'].unstack().T
    general_profit = ((df_auction_type['sell_price'].sum(axis=1).cumsum() - df_auction_type['buy_price'].sum(axis=1).cumsum()) / 10000).astype(int)

    if item:
        item_history[item].dropna(how='all').plot()
    elif date:
        general_profit.loc[date:].plot()

In [ ]:
get_profits(item='Mighty Rage Potion')

In [ ]:
get_profits(date='2020-02-15')

In [ ]:
add_item_prices()

In [ ]:
verbose = True

with open('intermediate/items_enriched.yaml', 'r') as f:
    user_items = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
pd.DataFrame.from_dict(user_items).T['min_list_price'].dropna()

In [ ]:
prices = pd.DataFrame.from_dict(user_items).T

In [ ]:
prices['lower_price'] = prices['buy_price'].fillna(prices['sell_price'])
prices['upper_price'] = prices['sell_price'].fillna(prices['buy_price'])

In [ ]:
prices = ((prices['upper_price'] + prices['lower_price']) / 2).dropna()

In [ ]:


ingredient_raw = pd.DataFrame.from_dict(user_items).T[['gross_profit', 'auction_success', 'made_from']].dropna()

ingredient_raw['ingredient_value'] = ingredient_raw['gross_profit'] * ingredient_raw['auction_success']

ingredient_df = pd.DataFrame()
for item, recipe in ingredient_raw['made_from'].items():
    for ingredient, count in recipe.items():
        ingredient_df.loc[item, ingredient] = count
        
xs = ingredient_df.drop(['Crystal Vial', 'Leaded Vial', 'Empty Vial'], axis=1).fillna(0).astype(int)
ys = ingredient_raw['ingredient_value']

In [ ]:
ingredient_raw

In [ ]:
def update_beta(pred_beta, y, x, learning_rate=0.02):
    actual_value = y * (x / x.sum())
    actual_value = actual_value[actual_value>0]

    new_beta = pred_beta.copy()
    new_beta.update(actual_value)

    # Introduce some truthiness to the betas
    pred_beta += ((new_beta - pred_beta) * learning_rate)
    return pred_beta

In [ ]:
import numpy as np

In [ ]:
random_start = ((np.random.rand(len(xs.columns)) - 0.5) + 1) * ys.mean()

In [ ]:
pred_beta = pd.Series(random_start, index=xs.columns)

In [ ]:
all_betas = pd.DataFrame(columns=pred_beta.index)

for i in range(800):
    for item, y in ys.sample(10).items():
        x = xs.loc[item]
        pred_beta = update_beta(pred_beta, y, x, learning_rate=0.005)
        
        if verbose:
            beta_proportion = ((pred_beta / pred_beta.sum()))
            all_betas.loc[i] = beta_proportion
if verbose:
    all_betas.plot(legend=False)
            
beta_proportion = ((pred_beta / pred_beta.sum()))            
beta_proportion.sort_values()         

In [ ]:
pred_beta

In [ ]:
# Need a POPULARITY metric, something that signifies how many of the last x auctions have been item